In [175]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
from mlxtend.regressor import StackingRegressor

In [176]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
from mlxtend.regressor import StackingRegressor

In [177]:
df=pd.read_excel(r'illite.xlsx')

In [178]:
df.head()

,Depth,B.Density,API,resis.90,DT-C,DT_SH,Illite from XRD
0,1827.3,2.614774,142.5504,83.8006,213.7932,354.1896,8.0
1,1828.5,2.620147,146.3731,88.2865,211.9861,352.3401,12.0
2,1829.5,2.608157,163.5804,92.1651,218.1440,359.6917,11.0
3,1830.8,2.611325,121.8629,90.2894,206.5182,343.3083,12.0
4,1831.9,2.624907,150.8556,88.9865,217.8588,357.3575,7.0


In [179]:
df.corr()

,Depth,B.Density,API,resis.90,DT-C,DT_SH,Illite from XRD
Depth,1.000000,-0.120963,0.198230,0.082312,-0.449168,0.191432,-0.158212
B.Density,-0.120963,1.000000,-0.422898,0.011434,-0.106291,-0.269532,0.264321
API,0.198230,-0.422898,1.000000,0.307387,0.041697,0.025347,-0.263613
resis.90,0.082312,0.011434,0.307387,1.000000,-0.224173,-0.181945,-0.127384
DT-C,-0.449168,-0.106291,0.041697,-0.224173,1.000000,-0.591326,-0.031190
DT_SH,0.191432,-0.269532,0.025347,-0.181945,-0.591326,1.000000,0.015535
Illite from XRD,-0.158212,0.264321,-0.263613,-0.127384,-0.031190,0.015535,1.000000


In [180]:
df.columns

Index(['Depth', 'B.Density', 'API', 'resis.90', 'DT-C', 'DT_SH',
       'Illite from XRD'],
      dtype='object')

In [199]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import r2_score, mean_squared_error
from mlxtend.regressor import StackingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
import numpy as np
from mlxtend.regressor import StackingCVRegressor

# Load your dataset (replace this with your actual data loading code)
# df = pd.read_csv('your_data.csv')
# selecting features and label
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

# Define features (X) and label (y)
X = df.iloc[:, :-1]  # Features
y = df.iloc[:, -1]   # Label

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=10)

# Apply Polynomial Features
degree = 2  # You can change the degree based on your requirement
poly = PolynomialFeatures(degree=degree, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Print new shape after polynomial transformation
X_train_poly.shape, X_test_poly.shape

# Initialize the base models
xgb = XGBRegressor()
gbr = GradientBoostingRegressor()

# Initialize the meta-model
meta_model = LinearRegression()
from mlxtend.regressor import StackingCVRegressor

stacked_model = StackingCVRegressor(regressors=(xgb, gbr),
                            meta_regressor=meta_model,cv=10)

# Fit the model on the entire training set
stacked_model.fit(X_train_poly, y_train)



StackingCVRegressor(cv=10, meta_regressor=LinearRegression(),
                    regressors=(XGBRegressor(base_score=None, booster=None,
                                             callbacks=None,
                                             colsample_bylevel=None,
                                             colsample_bynode=None,
                                             colsample_bytree=None, device=None,
                                             early_stopping_rounds=None,
                                             enable_categorical=False,
                                             eval_metric=None,
                                             feature_types=None, gamma=None,
                                             grow_policy=None,
                                             importance_type=None,
                                             interaction_constraints=None,
                                             learning_rate=None, max_bin=None,
                                             max_cat_threshold=None,
                                             max_cat_to_onehot=None,
                                             max_delta_step=None,
                                             max_depth=None, max_leaves=None,
                                             min_child_weight=None, missing=nan,
                                             monotone_constraints=None,
                                             multi_strategy=None,
                                             n_estimators=None, n_jobs=None,
                                             num_parallel_tree=None,
                                             random_state=None, ...),
                                GradientBoostingRegressor()))

In [200]:
import os

# Create the output/cnn directory if it doesn't exist
output_dir = "./output/stack_1"
os.makedirs(output_dir, exist_ok=True)

In [201]:
pred_2 = stacked_model.predict(X_test_poly)
pred_1 = stacked_model.predict(X_train_poly)


In [202]:
a = np.stack([y_train, pred_1], axis=1) # training
a = pd.DataFrame(a, columns=['y_train', 'y_train_pred'])

b = np.stack([y_test, pred_2], axis=1) # testing 
b = pd.DataFrame(b, columns=['y_test', 'y_test_pred'])



with pd.ExcelWriter("./output/stack_1/predicted_stack_1_model_GCV.xlsx") as writer:
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    a.to_excel(writer, sheet_name="training", index=True)
    b.to_excel(writer, sheet_name="testing", index=True)


In [203]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error

train_r2 = r2_score(y_train,pred_1) # r2_score for training set
train_mae = mean_absolute_error(y_train, pred_1) # mean absolute error for training set
train_mse = mean_squared_error(y_train, pred_1) # mean squared error for training set
train_mape = mean_absolute_percentage_error(y_train, pred_1) # mean_absolute_percentage_error for training set
train_ev = explained_variance_score(y_train, pred_1) 
train_maxE = max_error(y_train, pred_1)
train_minE = min(abs(y_train - pred_1))

test_r2 = r2_score(y_test, pred_2) # r2_score for testing set
test_mae = mean_absolute_error(y_test, pred_2) # mean absolute error for testing set
test_mse = mean_squared_error(y_test, pred_2) # mean squared error for testing set
test_mape = mean_absolute_percentage_error(y_test, pred_2) # mean_absolute_percentage_error for testing set
test_ev = explained_variance_score(y_test, pred_2) 
test_maxE = max_error(y_test, pred_2)
test_minE = min(abs(y_test - pred_2))

metrics = {
'performance_metrics': ['R2', 'MAE', 'MSE', 'MAPE', 'EV', 'maxE', 'minE'],
'training': [train_r2, train_mae, train_mse, train_mape, train_ev, train_maxE, train_minE],
'testing': [test_r2, test_mae, test_mse, test_mape, test_ev, test_maxE, test_minE],

}

performance_metrics = pd.DataFrame(metrics)
performance = performance_metrics.transpose()
performance.to_csv('./output/stack_1/performance_stack_1.csv')
performance

,0,1,2,3,4,5,6
performance_metrics,R2,MAE,MSE,MAPE,EV,maxE,minE
training,0.873732,1.515355,3.588378,0.213408,0.873766,5.279763,0.001288
testing,0.448345,3.002595,15.784292,0.336878,0.448412,11.643759,0.02697


In [204]:
#from sklearn.externals import joblib
from joblib import dump, load
dump(stacked_model, './output/stack_1/trained_stack_1_model.joblib')

#clf = load('trained_linear_regression_model.joblib')

['./output/stack_1/trained_stack_1_model.joblib']